In [21]:
import findspark
findspark.init('C:\spark-2.3.1-bin-hadoop2.7')

import pyspark
sc = pyspark.SparkContext(appName="myAppName")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=myAppName, master=local[*]) created by __init__ at <ipython-input-4-8eecae803b56>:2 

In [ ]:
# program przewidujący który z 3 wytwórców win we włoszech jest faktycznym 
# producentem na podstawie pozostałych cech wina

In [20]:
rawData = sc.textFile('../datasets/wine.data')

rawData

rawData.take(5)

['1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065',
 '1,13.2,1.78,2.14,11.2,100,2.65,2.76,.26,1.28,4.38,1.05,3.4,1050',
 '1,13.16,2.36,2.67,18.6,101,2.8,3.24,.3,2.81,5.68,1.03,3.17,1185',
 '1,14.37,1.95,2.5,16.8,113,3.85,3.49,.24,2.18,7.8,.86,3.45,1480',
 '1,13.24,2.59,2.87,21,118,2.8,2.69,.39,1.82,4.32,1.04,2.93,735']

In [22]:
#pierwsza kolumna to nr producenta 1,2,3 kolejne to pozostałe cechy

In [24]:
from pyspark.mllib.regression import LabeledPoint

def parsePoint(line):
    values = [float(x) for x in line.split(',')] #zamienia rawdata na liste
    return LabeledPoint(values[0], values[1:]) # zamienia liste na obiektlabeledpoint [nr_winnicy, reszta danych]

In [28]:
parsedData =  rawData.map(parsePoint) #mapuje sparsowane dane
parsedData


PythonRDD[13] at RDD at PythonRDD.scala:49

In [30]:
parsedData.take(5)

[LabeledPoint(1.0, [14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]),
 LabeledPoint(1.0, [13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0]),
 LabeledPoint(1.0, [13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0]),
 LabeledPoint(1.0, [14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0]),
 LabeledPoint(1.0, [13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0])]

In [64]:
(trainingData, testData) =parsedData.randomSplit([0.65, 0.35]) #podzial danych na trainingData i testData
trainingData


PythonRDD[178] at RDD at PythonRDD.scala:49

In [65]:
trainingData.take(5)

[LabeledPoint(1.0, [14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]),
 LabeledPoint(1.0, [13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0]),
 LabeledPoint(1.0, [14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0]),
 LabeledPoint(1.0, [13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0]),
 LabeledPoint(1.0, [14.2,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0])]

In [66]:
from pyspark.mllib.tree import DecisionTree

model = DecisionTree.trainClassifier(trainingData,
                                    numClasses=4,
                                    categoricalFeaturesInfo={},
                                    impurity = 'gini',
                                     maxDepth = 3,
                                     maxBins = 32)

In [67]:
predictions = model.predict(testData.map(lambda x:  x.features)) #przewidywanie 
predictions.take(5)

[1.0, 1.0, 1.0, 1.0, 1.0]

In [68]:
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
labelsAndPredictions.take(5)

[(1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]

In [69]:
testAcc = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(testData.count()) 
print('Dokładność testu: ' , testAcc)

Dokładność testu:  0.9076923076923077


In [70]:
from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(labelsAndPredictions)
metrics.accuracy

0.9076923076923077

In [71]:
metrics.precision(1.0)

1.0

In [72]:
metrics.confusionMatrix()
metrics.confusionMatrix().toArray()


array([[24.,  2.,  1.],
       [ 0., 26.,  0.],
       [ 0.,  3.,  9.]])